## Weighted-Sum Optimization Model

In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp

### Sets and Indices
- Let $F$ denote the set of flight legs (e.g., IST-YVR, YVR-IST, etc.), indexed by $i \in F$.
- Let $K$ denote the set of aircraft fleet types (e.g., B777-300ER, B787-9), indexed by $j \in K$.
- Let $P$ denote the set of paired (outbound, return) flight legs, where each pair is indexed by $(i, i') \in P$ (e.g., (TK75, TK76)).


### Parameters (Inputs)
- $d_i$: distance of flight leg $i$ (miles)
- $h_i$: block time of flight leg $i$ (hours), computed as
  $$
  h_i = \frac{d_i}{v}
  $$
  where $v$ is an assumed constant cruise speed
- $N_j$: number of available aircraft of fleet type $j$
- $\text{range}_j$: maximum feasible flight range of fleet type $j$ (miles)
- $H_{\max}$: maximum allowable crew-hours (block hours) per aircraft per day

**Fleet operating cost parameters**
- $\text{fuel}_j$: fuel burn rate of fleet type $j$ (kg/hour)
- $\text{crew}_j$: crew cost rate of fleet type $j$ (USD/hour)
- $\text{maint}_j$: maintenance and other operating cost rate of fleet type $j$ (USD/hour)
- $p_{\text{fuel}}$: fuel price (USD/kg)

The operating cost of assigning fleet $j$ to flight $i$ is defined as:
$$
c_{i,j}
=
\left(
\text{fuel}_j \cdot p_{\text{fuel}}
+
\text{crew}_j
+
\text{maint}_j
\right) h_i
$$

**Revenue model parameters (per flight)**
- $p^E_i, p^F_i, p^P_i, p^B_i$: prices of Economy, Flex, Prime, and Business seats on flight $i$
- $D^E_i, D^F_i, D^P_i, D^B_i$: demand limits for each fare class on flight $i$
- $\text{econcap}_j$: number of economy-class seats on fleet type $j$
- $\text{bizcap}_j$: number of business-class seats on fleet type $j$

In [ ]:
flights = pd.DataFrame([
    {"fid":"TK75", "orig":"IST", "dest":"YVR", "dist_mi":5973.9},
    {"fid":"TK76", "orig":"YVR", "dest":"IST", "dist_mi":5973.9},

    {"fid":"TK001", "orig":"IST", "dest":"JFK", "dist_mi":5000.0}, 
    {"fid":"TK002", "orig":"JFK", "dest":"IST", "dist_mi":5000.0},

    {"fid":"TK193", "orig":"IST", "dest":"LHR", "dist_mi":1550.0},
    {"fid":"TK194", "orig":"LHR", "dest":"IST", "dist_mi":1550.0},
    
    {"fid":"TK9", "orig":"IST", "dest":"LAX", "dist_mi":6852.0},
    {"fid":"TK10", "orig":"LAX", "dest":"IST", "dist_mi":6852.0},
    
    {"fid":"TK79", "orig":"IST", "dest":"SFO", "dist_mi":6705.2},
    {"fid":"TK80", "orig":"SFO", "dest":"IST", "dist_mi":6705.2},
    
    {"fid":"TK5", "orig":"IST", "dest":"ORD", "dist_mi":5473.6},
    {"fid":"TK6", "orig":"ORD", "dest":"IST", "dist_mi":5473.6},
    
    {"fid":"TK77", "orig":"IST", "dest":"MIA", "dist_mi":5967.4},
    {"fid":"TK78", "orig":"MIA", "dest":"IST", "dist_mi":5967.4},
    
    {"fid":"TK7", "orig":"IST", "dest":"IAD", "dist_mi":5225.1},
    {"fid":"TK8", "orig":"IAD", "dest":"IST", "dist_mi":5225.1},
    
    {"fid":"TK1821", "orig":"IST", "dest":"CDG", "dist_mi":1378.8},
    {"fid":"TK1822", "orig":"CDG", "dest":"IST", "dist_mi":1378.8},
    
    {"fid":"TK1587", "orig":"IST", "dest":"FRA", "dist_mi":1145.1},
    {"fid":"TK1588", "orig":"FRA", "dest":"IST", "dist_mi":1145.1},
    
    {"fid":"TK758", "orig":"IST", "dest":"DXB", "dist_mi":1882.6},
    {"fid":"TK759", "orig":"DXB", "dest":"IST", "dist_mi":1882.6},
    
    {"fid":"TK54", "orig":"IST", "dest":"SIN", "dist_mi":5394.3},
    {"fid":"TK55", "orig":"SIN", "dest":"IST", "dist_mi":5394.3},
    
    {"fid":"TK68", "orig":"IST", "dest":"BKK", "dist_mi":4665.0},
    {"fid":"TK69", "orig":"BKK", "dest":"IST", "dist_mi":4665.0},
    
    {"fid":"TK50", "orig":"IST", "dest":"NRT", "dist_mi":5593.4},
    {"fid":"TK51", "orig":"NRT", "dest":"IST", "dist_mi":5593.4},
    
    {"fid":"TK70", "orig":"IST", "dest":"HKG", "dist_mi":4983.8},
    {"fid":"TK71", "orig":"HKG", "dest":"IST", "dist_mi":4983.8},
    
    {"fid":"TK42", "orig":"IST", "dest":"JNB", "dist_mi":4636.6},
    {"fid":"TK43", "orig":"JNB", "dest":"IST", "dist_mi":4636.6},
    
    {"fid":"TK174", "orig":"IST", "dest":"SYD", "dist_mi":9300.6},
    {"fid":"TK175", "orig":"SYD", "dest":"IST", "dist_mi":9300.6}
])
# ref: www.airmilescalculator.com


fleet = pd.DataFrame([
    {
        "type":"B777-300ER", 
        "seats":349, 
        "fuel_kg_hr":7500,
        "maint_usd_hr":2200,
        "pilot_usd_hr":1800,
        "cabin_usd_hr":1200,
        "N":2, "range_mi":7600
    },
    {
        "type":"B787-9", 
        "seats":290, 
        "fuel_kg_hr":5600,
        "maint_usd_hr":2000,
        "pilot_usd_hr":1700,
        "cabin_usd_hr":1100,
        "N":3, "range_mi":7600
    },
    {
        "type": "A350-900",
        "seats": 440,
        "fuel_kg_hr": 7650,
        "maint_usd_hr": 2200,
        "pilot_usd_hr": 1800,
        "cabin_usd_hr": 1200,
        "N":3, "range_mi": 9551
    },
    {
        "type": "A330-300",
        "seats": 289,
        "fuel_kg_hr": 5800,
        "maint_usd_hr": 1900,
        "pilot_usd_hr": 1700,
        "cabin_usd_hr": 1100,
        "N": 4,
        "range_mi": 7301
    },
    {
        "type": "A330-200",
        "seats": 250,
        "fuel_kg_hr": 5500,
        "maint_usd_hr": 1800,
        "pilot_usd_hr": 1700,
        "cabin_usd_hr": 1000,
        "N": 3,
        "range_mi": 3480
    },
    {
        "type": "A321neo",
        "seats": 190,
        "fuel_kg_hr": 2400,
        "maint_usd_hr": 1200,
        "pilot_usd_hr": 1400,
        "cabin_usd_hr": 700,
        "N": 5,
        "range_mi": 4598
    }
])

In [3]:
FUEL_PRICE_USD_PER_KG = 0.90
CRUISE_MPH = 560

# daily limit per aircraft
H_MAX = 15.0  # 15 hours/day/aircraft

# flight hours
flights["hours"] = flights["dist_mi"] / CRUISE_MPH

# cost matrix c[i,j]
I, J = len(flights), len(fleet)
C = np.zeros((I, J))

for i, f in flights.iterrows():
    for j, a in fleet.iterrows():
        hours = f["hours"]

        fuel_cost = a["fuel_kg_hr"] * hours * FUEL_PRICE_USD_PER_KG
        maint_cost = a["maint_usd_hr"] * hours
        crew_cost = (a["pilot_usd_hr"] + a["cabin_usd_hr"]) * hours

        C[i, j] = fuel_cost + maint_cost + crew_cost

In [4]:
# Per-flight revenue inputs (simple table)
rev = flights[["fid"]].copy()

# Example: same fare prices/demands for all flights (easy baseline)
# Later: customize row-by-row for different long-haul routes
rev["Eco_p"],   rev["Eco_d"]   = 635, 180
rev["Flex_p"],  rev["Flex_d"]  = 765, 140
rev["Prime_p"], rev["Prime_d"] = 920, 80
rev["Bus_p"],   rev["Bus_d"]   = 2300, 49

Eco_p  = rev["Eco_p"].values
Flex_p = rev["Flex_p"].values
Prime_p= rev["Prime_p"].values
Bus_p  = rev["Bus_p"].values

Eco_d  = rev["Eco_d"].values
Flex_d = rev["Flex_d"].values
Prime_d= rev["Prime_d"].values
Bus_d  = rev["Bus_d"].values


We should work on the above price/demand quantities. 

Maybe have demand proportional to distance flown? Need a way to vary demand with destination because tickets sold becomes "too" unrealistic

In [5]:
# Cabin capacities by fleet type
# Example:
fleet.loc[fleet["type"] == "B777-300ER", "biz_seats"] = 49
fleet.loc[fleet["type"] == "B777-300ER", "econ_seats"] = 300

fleet.loc[fleet["type"] == "B787-9", "biz_seats"] = 30
fleet.loc[fleet["type"] == "B787-9", "econ_seats"] = 270

biz_cap  = fleet["biz_seats"].values.astype(float)
econ_cap = fleet["econ_seats"].values.astype(float)


### Decision Variables

**Fleet assignment**
- $x_{i,j} \in \{0,1\}$  
  $x_{i,j} = 1$ if flight $i$ is assigned fleet type $j$, and $0$ otherwise.

**Seat allocation (per flight)**
- $y^E_i, y^F_i, y^P_i \in \mathbb{Z}_{\ge 0}$: number of Economy, Flex, and Prime seats sold on flight $i$
- $y^B_i \in \mathbb{Z}_{\ge 0}$: number of Business seats sold on flight $i$

In [6]:
I, J = len(flights), len(fleet)

x = cp.Variable((I, J), boolean=True)

Eco   = cp.Variable(I, integer=True)
Flex  = cp.Variable(I, integer=True)
Prime = cp.Variable(I, integer=True)
Bus   = cp.Variable(I, integer=True)

### Objective Function (Weighted-Sum Formulation)

The goal is to simultaneously maximize total revenue and minimize total operating cost.  
Using a weighted-sum approach, the objective is:
$$
\max
\;
\sum_{i \in F}
\left(
p^E_i y^E_i
+
p^F_i y^F_i
+
p^P_i y^P_i
+
p^B_i y^B_i
\right)
\;
-\;
\lambda
\sum_{i \in F}
\sum_{j \in K}
c_{i,j} x_{i,j}
$$

where $\lambda > 0$ is a weighting parameter controlling the trade-off between revenue maximization and cost minimization.

In [7]:
total_revenue = cp.sum(cp.multiply(Eco_p, Eco) +
                       cp.multiply(Flex_p, Flex) +
                       cp.multiply(Prime_p, Prime) +
                       cp.multiply(Bus_p, Bus))

total_cost = cp.sum(cp.multiply(C, x))

#LAMBDA = 0.005
#objective = cp.Maximize(total_revenue - LAMBDA * total_cost)
objective = cp.Maximize(total_revenue - total_cost)

After doing this and running it, I'm not exactly sure why we need lambda or how it can be interpreted in terms of the problem context. Shouldn't the costs being produced stay fixed after all these calculations?

### Constraints

#### 1. Flight Coverage
Each flight must be assigned exactly one fleet type:
$$
\sum_{j \in K} x_{i,j} = 1
\quad \forall i \in F
$$


#### 2. Crew-Hour (Block-Hour) Availability
Total crew-hours consumed by fleet type $j$ cannot exceed available crew-hours:
$$
\sum_{i \in F} h_i x_{i,j}
\le
N_j \cdot H_{\max}
\quad \forall j \in K
$$


#### 3. Aircraft Range Feasibility
Fleet types may only be assigned to routes within their operational range:
$$
x_{i,j} = 0
\quad \text{if } d_i > \text{range}_j
$$


#### 4. Operational Pairing (Round-Trip Consistency)
For paired outbound and return flight legs $(i,i') \in P$, the same fleet type must be used:
$$
x_{i,j} = x_{i',j}
\quad \forall j \in K,\; \forall (i,i') \in P
$$


#### 5. Demand Constraints (Seat Allocation)
Seat sales on each flight may not exceed demand:
$$
0 \le y^E_i \le D^E_i,\quad
0 \le y^F_i \le D^F_i,\quad
0 \le y^P_i \le D^P_i,\quad
0 \le y^B_i \le D^B_i
\quad \forall i \in F
$$


#### 6. Capacity Linking Constraints
Seat sales on each flight must fit within the cabin capacities implied by the assigned fleet type:
$$
y^B_i
\le
\sum_{j \in K} \text{bizcap}_j x_{i,j}
\quad \forall i \in F
$$

$$
y^E_i + y^F_i + y^P_i
\le
\sum_{j \in K} \text{econcap}_j x_{i,j}
\quad \forall i \in F
$$

In [8]:
constraints = []

# (A) Flight cover
constraints += [cp.sum(x[i, :]) == 1 for i in range(I)]

# (B) Pairing (if your pairs exist)
pairs = [("TK75", "TK76"), ("TK001", "TK002"), ("TK193", "TK194")]
fid_to_idx = {fid: idx for idx, fid in enumerate(flights["fid"])}

for f1, f2 in pairs:
    i1, i2 = fid_to_idx[f1], fid_to_idx[f2]
    constraints += [x[i1, :] == x[i2, :]]

# (C) Crew-hours availability (Option A)
CREW_H_MAX = H_MAX
for j in range(J):
    crew_hours_used_j = cp.sum(cp.multiply(flights["hours"].values, x[:, j]))
    constraints += [crew_hours_used_j <= int(fleet.loc[j, "N"]) * CREW_H_MAX]

# (D) Range feasibility
for i, f in flights.iterrows():
    for j, a in fleet.iterrows():
        if float(f["dist_mi"]) > float(a["range_mi"]):
            constraints += [x[i, j] == 0]

# (E) Demand constraints (per flight)
constraints += [
    Eco >= 0, Flex >= 0, Prime >= 0, Bus >= 0,
    Eco <= Eco_d,
    Flex <= Flex_d,
    Prime <= Prime_d,
    Bus <= Bus_d
]

# (F) Capacity link: seats sold must fit cabin capacities implied by assigned fleet
for i in range(I):
    constraints += [
        Bus[i] <= biz_cap @ x[i, :],
        Eco[i] + Flex[i] + Prime[i] <= econ_cap @ x[i, :],
    ]


### Solving the model + summary
This formulation represents a mixed-integer linear program (MILP) that integrates fleet assignment and revenue management across multiple international flights using a weighted-sum multi-objective approach.

In [9]:
prob = cp.Problem(objective, constraints)
prob.solve()

1450215.442857143

In [10]:
print("Status:", prob.status)
print("Weighted objective (Profit):", prob.value)
print("Total revenue (USD):", float(total_revenue.value))
print("Total cost (USD):", float(total_cost.value))
print()

assignments = []
for i, f in flights.iterrows():
    j_star = int(np.argmax(x.value[i, :]))

    # capacities implied by assigned fleet
    econ_capacity = float(fleet.loc[j_star, "econ_seats"])
    biz_capacity  = float(fleet.loc[j_star, "biz_seats"])
    total_capacity = econ_capacity + biz_capacity

    eco_sold   = float(Eco.value[i])
    flex_sold  = float(Flex.value[i])
    prime_sold = float(Prime.value[i])
    biz_sold   = float(Bus.value[i])

    econ_sold = eco_sold + flex_sold + prime_sold
    total_sold = econ_sold + biz_sold

    rev_i = float(Eco_p[i]*eco_sold + Flex_p[i]*flex_sold + Prime_p[i]*prime_sold + Bus_p[i]*biz_sold)
    cost_i = float(C[i, j_star])

    assignments.append({
        "Flight": f["fid"],
        "Route": f"{f['orig']}-{f['dest']}",
        "Fleet": fleet.loc[j_star, "type"],
        "Hours": round(float(f["hours"]), 2),

        "Econ Sold/Cap": f"{int(round(econ_sold))}/{int(econ_capacity)}",
        "Biz Sold/Cap":  f"{int(round(biz_sold))}/{int(biz_capacity)}",
        "Total Sold/Cap":f"{int(round(total_sold))}/{int(total_capacity)}",

        "Revenue ($)": round(rev_i, 0),
        "Cost ($)": round(cost_i, 0),
        "Profit ($)": round (rev_i - cost_i, 0)
    })

df_out = pd.DataFrame(assignments)

df_out = df_out[[
    "Flight","Route","Fleet","Hours",
    "Econ Sold/Cap","Biz Sold/Cap","Total Sold/Cap",
    "Revenue ($)","Cost ($)", "Profit ($)"
]]

print(df_out.to_string(index=False))


Status: optimal
Weighted objective (Profit): 1450215.442857143
Total revenue (USD): 1939700.0
Total cost (USD): 489484.5571428571

Flight   Route      Fleet  Hours Econ Sold/Cap Biz Sold/Cap Total Sold/Cap  Revenue ($)  Cost ($)  Profit ($)
  TK75 IST-YVR     B787-9  10.67       270/270        30/30        300/300     281450.0  104970.0    176480.0
  TK76 YVR-IST     B787-9  10.67       270/270        30/30        300/300     281450.0  104970.0    176480.0
 TK001 IST-JFK B777-300ER   8.93       300/300        49/49        349/349     344200.0  106696.0    237504.0
 TK002 JFK-IST B777-300ER   8.93       300/300        49/49        349/349     344200.0  106696.0    237504.0
 TK193 IST-LHR B777-300ER   2.77       300/300        49/49        349/349     344200.0   33076.0    311124.0
 TK194 LHR-IST B777-300ER   2.77       300/300        49/49        349/349     344200.0   33076.0    311124.0
